In [2]:
import poppy
import jax
import jax.numpy as jnp
import functools
import galois
import numpy as np

In [5]:
R = jnp.arange(4)
jnp.where(R[:,None]-R[None,:] == 0, 0, 1)

Array([[0, 1, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 0]], dtype=int64, weak_type=True)

In [8]:
N  = 10000
d  = 2
p  = 7
p1 = 13
p2 = 17
n  = 6
q  = p ** n
f  = poppy.field( p,  n )
F  = poppy.field( p1, d )
FF = poppy.field( p2, n )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 0 ), ( N, F.n, F.n ),            0,       q, dtype = poppy.DTYPE )
AA = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ), ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = poppy.DTYPE )
BB = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ), ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = poppy.DTYPE )
A  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ), ( FF.n, FF.n ),             0, 2 ** 32, dtype = poppy.DTYPE )
B  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ), ( FF.n, FF.n ),             0, 2 ** 32, dtype = poppy.DTYPE )

@jax.jit
def poppyhash( a ):
    b = poppy.proj( AA @ a + BB, F )
    c = poppy.proj( A @ b + B, FF )
    return c % ( 4 * N )

liftbatch = jax.vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = jax.vmap( poppyhash )

ap = liftbatch( a, f )
h  = hashbatch( ap ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu ), F.q, FF.q

(10000, 8832, 169, 24137569)